<a href="https://colab.research.google.com/github/ghirailghiro/GPU_Computing_Project/blob/8-fine-tuning-and-experiments/Gradient_First_Step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Cuda plugin

!pip install nvcc4jupyter
%load_ext nvcc4jupyter
!nvidia-smi

# plugin for cpp sintax highlighting

!wget -O cpp_plugin.py https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
%load_ext cpp_plugin

Source files will be saved in "/tmp/tmpftefhjn3".
Sat Mar  9 21:14:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------

In [2]:
!sudo apt update
!sudo apt install -y build-essential
!sudo apt install -y libopencv-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,076 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,994 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,561 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jamm

In [3]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!ls '/content/drive/My Drive/GPU Computing/human detection dataset'

0  1


In [11]:
%%writefile gradient_computation.cu
#include <opencv2/opencv.hpp>
#include <fstream>
#include <sstream>
#include <iostream>
#include <vector>
#include <filesystem>

namespace fs = std::filesystem;

void saveDescriptorAsCSVHeader(const std::vector<int>& descriptor, const std::string& filename, const std::string& label) {
    std::ofstream file(filename, std::ios::app);
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file " << filename << " for writing." << std::endl;
        return;
    }

    // Write the descriptor to the CSV file
    for (size_t i = 0; i < descriptor.size(); ++i) {
        file << "x" <<descriptor[i];
        if (i < descriptor.size() - 1) {
            file << ",";
        }
    }
    file << "," << label << "\n";
    file.close();
}


void saveDescriptorAsCSV(const std::vector<float>& descriptor, const std::string& filename, int label) {
    std::ofstream file(filename, std::ios::app);
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file " << filename << " for writing." << std::endl;
        return;
    }

    // Write the descriptor to the CSV file
    for (size_t i = 0; i < descriptor.size(); ++i) {
        file << descriptor[i];
        if (i < descriptor.size() - 1) {
            file << ",";
        }
    }
    file << "," << label << "\n";
    file.close();
}

__global__ void computeGradients(unsigned char* image, float *d_magnitude, float *d_orientation, float *d_histograms, int width, int height, int cellSize) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int idy = blockIdx.y * blockDim.y + threadIdx.y;
    int indexCurrent = idy * width + idx;

    if (idx >= width || idy >= height) return; // Boundary check

    float G_x = 0;
    if (idx > 0 && idx < width - 1) {
        G_x = (float)image[idy * width + (idx + 1)] - (float)image[idy * width + (idx - 1)];
    }

    float G_y = 0;
    if (idy > 0 && idy < height - 1) {
        G_y = (float)image[(idy + 1) * width + idx] - (float)image[(idy - 1) * width + idx];
    }

    d_magnitude[indexCurrent] = sqrtf(G_x * G_x + G_y * G_y);
    d_orientation[indexCurrent] = atan2f(G_y, G_x);

    // Compute histogram bin for the current gradient
    int cellX = idx / cellSize;
    int cellY = idy / cellSize;
    int histIndex = cellY * (width / cellSize) + cellX;
    int numBins = 9; // Assuming 9 orientation bins
    float binWidth = M_PI / numBins;
    int bin = floor((d_orientation[indexCurrent] + M_PI) / binWidth);
    if (bin == numBins) bin = 0; // Wrap around
    atomicAdd(&d_histograms[histIndex * numBins + bin], d_magnitude[indexCurrent]);

    /*if((idx == 2 && idy == 2) || (idx==300 & idy == 300)){
      printf("Value for idx %d e idy %d - bin value: %d - orientation value %f \n", idx, idy, bin,d_orientation[indexCurrent] + M_PI);
    }*/
}

std::vector<float> computeDescriptors(const std::string& image_path){
        // Example: Load an image using OpenCV
    cv::Mat imageBeforeResize = cv::imread(image_path, cv::IMREAD_GRAYSCALE);
    cv::Mat image;
    cv::resize(imageBeforeResize, image, cv::Size(448, 448)); // Resize to standard size
    if(image.empty()) {
        std::cerr << "Failed to load image." << std::endl;
        return std::vector<float>();
    }

    unsigned char* d_image;
    size_t imageSize = image.total() * image.elemSize();
    cudaMalloc(&d_image, imageSize);
    cudaMemcpy(d_image, image.data, imageSize, cudaMemcpyHostToDevice);
    size_t sizeInBytes = image.total() * sizeof(float);
    float* d_magnitude;
    cudaError_t status = cudaMalloc((void **)&d_magnitude, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error (e.g., printing an error message and exiting)
        fprintf(stderr, "cudaMalloc failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    status = cudaMemset(d_magnitude, 0, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMemset failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    sizeInBytes = image.total() * sizeof(float);
    float* d_orientation;
    status = cudaMalloc((void **)&d_orientation, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error (e.g., printing an error message and exiting)
        fprintf(stderr, "cudaMalloc failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    status = cudaMemset(d_orientation, 0, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMemset failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }

    // Assuming image dimensions are reasonable for a grid of threads
    dim3 blockSize(16, 16);
    dim3 gridSize((image.cols + blockSize.x - 1) / blockSize.x,
                  (image.rows + blockSize.y - 1) / blockSize.y);

    // Allocate memory for histograms
    int cellSize = 128;
    int numCellsX = image.cols / cellSize;
    int numCellsY = image.rows / cellSize;
    size_t histSize = numCellsX * numCellsY * 9 * sizeof(float);
    float* d_histograms;
    status = cudaMalloc((void **)&d_histograms, histSize);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMalloc failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    status = cudaMemset(d_histograms, 0, histSize);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMemset failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }

    // Launch the kernel
    computeGradients<<<gridSize, blockSize>>>(d_image, d_magnitude, d_orientation, d_histograms, image.cols, image.rows, cellSize);

    cudaDeviceSynchronize();

    // Transfer histogram data from device to host
    float* h_histograms = new float[numCellsX * numCellsY * 9];
    cudaMemcpy(h_histograms, d_histograms, histSize, cudaMemcpyDeviceToHost);

    /*for (int i = 0; i < numCellsX * numCellsY; ++i) {
        for (int j = 0; j < 9; ++j) {
            std::cout << "Cell " << i << ", Bin " << j << ": " << h_histograms[i * 9 + j] << std::endl;
        }
    }*/
    // Cleanup

    /*// Normalization
    for (int i = 0; i < numCellsX * numCellsY; ++i) {
        float sum = 0.0f;
        for (int j = 0; j < 9; ++j) {
            sum += h_histograms[i * 9 + j] * h_histograms[i * 9 + j];
        }
        sum = sqrtf(sum);
        for (int j = 0; j < 9; ++j) {
            h_histograms[i * 9 + j] /= (sum + 1e-6); // Small constant added to avoid division by zero
        }
    }

    // Block Formation and Descriptor Computation
    std::vector<float> descriptor;
    for (int i = 0; i < numCellsY - 1; ++i) {
        for (int j = 0; j < numCellsX - 1; ++j) {
            // Concatenate histograms of four cells into a block
            for (int y = i; y < i + 2; ++y) {
                for (int x = j; x < j + 2; ++x) {
                    for (int k = 0; k < 9; ++k) {
                        descriptor.push_back(h_histograms[(y * numCellsX + x) * 9 + k]);
                    }
                }
            }
        }
    }*/
    // Block Formation and Descriptor Computation
    std::vector<float> descriptor;
    for (int i = 0; i < numCellsY - 1; ++i) {
        for (int j = 0; j < numCellsX - 1; ++j) {
            // Concatenate histograms of four cells into a block
            float blockSum = 0.0f;
            for (int y = i; y < i + 2; ++y) {
                for (int x = j; x < j + 2; ++x) {
                    for (int k = 0; k < 9; ++k) {
                        descriptor.push_back(h_histograms[(y * numCellsX + x) * 9 + k]);
                        blockSum += h_histograms[(y * numCellsX + x) * 9 + k] * h_histograms[(y * numCellsX + x) * 9 + k];
                    }
                }
            }
            // Block normalization
            float blockMagnitude = sqrtf(blockSum + 1e-6); // Small constant added to avoid division by zero
            for (int y = 0; y < 2; ++y) {
                for (int x = 0; x < 2; ++x) {
                    for (int k = 0; k < 9; ++k) {
                        descriptor[(i * (numCellsX - 1) + j) * 36 + (y * 2 + x) * 9 + k] /= blockMagnitude;
                    }
                }
            }
        }
    }
    cudaFree(d_image);
    cudaFree(d_magnitude);
    cudaFree(d_orientation);
    delete[] h_histograms;
    cudaFree(d_histograms);

    return descriptor;
}

int main() {

    std::string folder_path = "/content/drive/My Drive/GPU Computing/human detection dataset/1"; // Change this to your folder path
    std::vector<int> header;
    //1296, 6084, 144
    for (int i=1; i <= 144; ++i){
      header.push_back(i);
    }
    saveDescriptorAsCSVHeader(header, "descriptor.csv", "label");
    header.clear();
    //People present class
    //int count = 0;
    for (const auto& entry : fs::directory_iterator(folder_path)) {
        std::string file_path = entry.path().string();
        std::cout << "Processing image: " << file_path << std::endl;
        /*if(count == 4){
           break;
        }
        count++;*/

        std::vector<float> descriptor = computeDescriptors(file_path);
        if (descriptor.empty()) {
            std::cout << "Vector is empty" << std::endl;
        } else {
            int label = 1;
            std::vector<float> descriptor1;
            int length = static_cast<int>(descriptor.size());
            std::cout << "Dimension of descriptor: " << length << std::endl;
            descriptor1.push_back(length);
            saveDescriptorAsCSV(descriptor, "descriptor.csv", label);
            descriptor.clear(); // Clear the vector
            descriptor1.clear();
        }
    }
    //return 0;

      //Not people present class
      folder_path = "/content/drive/My Drive/GPU Computing/human detection dataset/0";
      for (const auto& entry : fs::directory_iterator(folder_path)) {
        std::string file_path = entry.path().string();
        std::cout << "Processing image: " << file_path << std::endl;


        std::vector<float> descriptor = computeDescriptors(file_path);
        if (descriptor.empty()) {
            std::cout << "Vector is empty" << std::endl;
        } else {
            int label = 0;
            std::vector<float> descriptor1;
            int length = static_cast<int>(descriptor.size());
            std::cout << "Dimension of descriptor: " << length << std::endl;
            descriptor1.push_back(length);
            saveDescriptorAsCSV(descriptor, "descriptor.csv", label);
            descriptor.clear(); // Clear the vector
            descriptor1.clear();
        }

    }


    return 0;
}


Overwriting gradient_computation.cu


In [12]:
!nvcc gradient_computation.cu -o gradient_computation `pkg-config --cflags --libs opencv4` -run


/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [42]:
!./gradient_computation

Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/110.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/111.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/100.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/108.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/105.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/118.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/1.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/115.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Comp

In [7]:
import os

def count_files(folder_path):
  """Counts the number of files in a directory (folder).

  Args:
      folder_path: The path to the directory.

  Returns:
      The number of files in the directory.
  """
  num_files = 0
  for entry in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, entry)):
      num_files += 1
  return num_files

# Example usage
folder_path = "/content/drive/My Drive/GPU Computing/human detection dataset/1"
number_of_files = count_files(folder_path)
print(f"There are {number_of_files} files in the folder.")

folder_path = "/content/drive/My Drive/GPU Computing/human detection dataset/0"
number_of_files1 = count_files(folder_path)
print(f"There are {number_of_files} files in the folder.")

print(f"Sum files {number_of_files + number_of_files1 }")

There are 559 files in the folder.
There are 559 files in the folder.
Sum files 921


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

# Assuming df is your DataFrame with features and labels
# Splitting data into features (X) and labels (y)

df = pd.read_csv("descriptor.csv")
new_value = 0

df.iloc[-1, df.columns.get_loc('label')] = new_value

X = df.drop(columns=['label'])  # Replace 'target_column' with the name of your target column
y = df['label']


In [14]:
df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x1288,x1289,x1290,x1291,x1292,x1293,x1294,x1295,x1296,label
0,0.160082,0.054192,0.041405,0.069380,0.120882,0.086396,0.083003,0.035284,0.051687,0.127215,...,0.380940,0.459166,0.196222,0.083245,0.109103,0.076047,0.092650,0.165129,0.101960,1
1,0.155204,0.021428,0.027436,0.075694,0.158440,0.034686,0.031321,0.028911,0.055047,0.181631,...,0.247135,0.141044,0.155049,0.117260,0.113964,0.115955,0.105155,0.183720,0.195014,1
2,0.008182,0.000604,0.005726,0.003646,0.109508,0.006432,0.004805,0.000743,0.000000,0.009506,...,0.087212,0.061066,0.070270,0.122547,0.517327,0.184480,0.131823,0.107885,0.064557,1
3,0.369105,0.116678,0.078077,0.165637,0.152847,0.034457,0.029996,0.032921,0.103930,0.224998,...,0.280176,0.344796,0.251098,0.204512,0.174569,0.142345,0.183038,0.213131,0.171418,1
4,0.205978,0.034457,0.033277,0.122893,0.361235,0.085854,0.047931,0.043924,0.075006,0.318118,...,0.088933,0.118565,0.192841,0.217345,0.287531,0.125397,0.100210,0.129198,0.069151,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,0.152358,0.048086,0.049918,0.061153,0.171713,0.047746,0.072496,0.074133,0.077988,0.176410,...,0.187247,0.204530,0.067668,0.082810,0.130152,0.072626,0.084566,0.161842,0.289210,0
917,0.228817,0.033902,0.033369,0.177762,0.085921,0.033367,0.048473,0.041711,0.458120,0.208474,...,0.130746,0.141633,0.115539,0.106635,0.396732,0.075105,0.080533,0.049667,0.021814,0
918,0.162489,0.097658,0.091273,0.249945,0.079594,0.039849,0.052861,0.155340,0.305876,0.241746,...,0.095122,0.016552,0.043277,0.043521,0.270326,0.035875,0.042821,0.019668,0.016011,0
919,0.016625,0.000000,0.000253,0.000794,0.055900,0.018411,0.005143,0.000907,0.000094,0.067652,...,0.314397,0.048032,0.034625,0.121201,0.135876,0.102511,0.144003,0.081457,0.139489,0


In [ ]:
y

0    1
1    1
2    1
3    0
Name: label, dtype: int64

In [15]:
# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training a Linear Support Vector Machine
svm = LinearSVC()
svm.fit(X_train, y_train)

# Making predictions
y_pred = svm.predict(X_test)

# Evaluating accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6864864864864865
